In [1]:
import torch
from helper.model import *
from helper.utils import *
from helper.distance_map import *
from helper.evaluate import *
import pandas as pd
import warnings

In [2]:
args_train_data = "uniref1/uniref1_train_split_0"
args_test_data = "uniref1/uniref1_test_split_0_curate"
args_model_name = "uniref1/uniref1_split0_triplet"
 
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
dtype = torch.float32
# load id ec from tran and test
id_ec_train, ec_id_dict_train = get_ec_id_dict(
    './data/' + args_train_data + '.csv')
id_ec_test, _ = get_ec_id_dict(
    './data/' + args_test_data + '.csv')

# load model
if False:
    # no model used for pretrained embedding
    model = lambda *args: args[0]
else:
    model = LayerNormNet(512, 128, device, dtype)
    checkpoint = torch.load('./model/' + args_model_name + '.pth')
    model.load_state_dict(checkpoint)

# compute distance map
emb_train = model_embedding_test(id_ec_train, model, device, dtype)
emb_test = model_embedding_test(id_ec_test, model, device, dtype)

In [3]:
update_step = 4000
eval_dist = {}
start = 0
end = update_step
while start < len(id_ec_test):
    eval_dist = update_dist_dict_blast(
        emb_test, emb_train, eval_dist, start, end, id_ec_test, id_ec_train)
    start = end
    if end + update_step <= len(id_ec_test): 
        end = end + update_step 
    else:
        end = len(id_ec_test)
    # release gpu memeory
    torch.cuda.empty_cache()

1313it [00:04, 321.89it/s]


In [4]:
eval_df = pd.DataFrame.from_dict(eval_dist)
# write the top 10 closest EC to _top10.csv
out_filename = './eval/' + args_test_data
# _ = write_top10_choices(eval_df, out_filename)
# maximum separation results
write_max_sep_choices(eval_df, out_filename,
                        first_grad=True,
                        use_max_grad=False)
# get preds and true labels
pred_label = get_pred_labels(out_filename, pred_type='_maxsep')
true_label, all_label = get_true_labels('./data/'+args_test_data)
pre, rec, f1, roc, acc = get_eval_metrics(
    pred_label, true_label, all_label)
print("############ EC calling results using maximum separation ############")
print('-' * 75)
print(f'>>> total samples: {len(true_label)} | total ec {len(all_label)} |\n'
        f'precision | recall | F1 | AUC | accuracy' )
print( f'{pre:.5} , {rec:.5} , {f1:.5} , {roc:.5} , {acc:.5}')
print('-' * 75)

c:\Users\canal\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:986: UserWarning: unknown class(es) ['3.1.26.11'] will be ignored
  warnings.warn('unknown class(es) {0} will be ignored'
c:\Users\canal\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:986: UserWarning: unknown class(es) ['1.14.19.9'] will be ignored
  warnings.warn('unknown class(es) {0} will be ignored'
c:\Users\canal\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:986: UserWarning: unknown class(es) ['2.6.1.80'] will be ignored
  warnings.warn('unknown class(es) {0} will be ignored'
c:\Users\canal\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:986: UserWarning: unknown class(es) ['4.2.2.10'] will be ignored
  warnings.warn('unknown class(es) {0} will be ignored'
c:\Users\canal\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:986: UserWarning: unknown class(es) ['4.1.1.36'] will be ignored
  warnings.warn('unknown class(es) {0} will be ignored'
c:\Users\canal\ana

############ EC calling results using maximum separation ############
---------------------------------------------------------------------------
>>> total samples: 1313 | total ec 653 |
precision | recall | F1 | AUC | accuracy
0.60533 , 0.64046 , 0.60546 , 0.81896 , 0.58492
---------------------------------------------------------------------------


python -u .\eval_maxsep_BLAST.py -tr uniref100/uniref100_train_split_0 -te uniref100/uniref100_test_split_0_curate -n uniref100/uniref100_split_0_triplet >> weighted.out; python -u .\eval_maxsep_BLAST.py -tr uniref100/uniref100_train_split_1 -te uniref100/uniref100_test_split_1_curate -n uniref100/uniref100_split_1_triplet >> weighted.out; python -u .\eval_maxsep_BLAST.py -tr uniref100/uniref100_train_split_2 -te uniref100/uniref100_test_split_2_curate -n uniref100/uniref100_split_2_triplet >> weighted.out; python -u .\eval_maxsep_BLAST.py -tr uniref100/uniref100_train_split_3 -te uniref100/uniref100_test_split_3_curate -n uniref100/uniref100_split_3_triplet >> weighted.out; python -u .\eval_maxsep_BLAST.py -tr uniref100/uniref100_train_split_4 -te uniref100/uniref100_test_split_4_curate -n uniref100/uniref100_split_4_triplet >> weighted.out